In [ ]:
## install the pytorch3d package; hit 'shift+enter' keys to run
!conda create --name pytorch3d --yes
!conda activate pytorch3d
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch --yes
!conda install -c conda-forge -c takatosp1 fvcore --yes
!conda install pytorch3d -c pytorch3d --yes
!python3 -m pip install PyGEL3D tqdm pandas scipy opencv-python 
!python3 -m pip install --upgrade matplotlib

In [ ]:
## download the concerthall optimization code base; hit 'shift+enter' keys to run
!git clone https://github.com/alexacarlson/concerthall.git /storage/concerthall

In [ ]:
## Train the graph convolutional neural network; hit 'shift+enter' keys to run
## NOTES on Training the Graph Convolutional Neural network to map 3D meshes --> audio quality parameters:
## To train your own graph convolutional network, you first need to create a folder in paperspace storage 
## at `/storage/concert_dataset` upload your data there. All of your meshes should be `.obj` files and
## should exist in a subdirectory of the dataset: `/storage/concert_dataset/OBJdatabase/`. 
## You will need to have a csv file of acoustic paramters (where each line is in the format `mesh.obj acoust_param1 acoust_param2 ... acoust_param10`) 
## located at and named `/storage/concert_dataset/AcousticParameters.csv`.
## The results (network weights and training loss values) will be saved to `/storage/mesh2acoustic_training_results`.
##
## Note that this function takes in the absolute filepath of a configuration file (CFG_PATH) that sets up the network architecture.
## To train on 8 quality metric labels:
#!cd /storage/concerthall && bash run_train_graphconv_regression.sh "/storage/concerthall/config/mesh2audioparams_8class.yml"
## To train on 10 quality metric labels:
!cd /storage/concerthall && bash run_train_graphconv_regression.sh "/storage/concerthall/config/mesh2audioparams_train.yml"

In [ ]:
## run the optimization framework; hit 'shift+enter' keys to run
## IMPORTANT INFO about running the acoustic optimization script:
## The Command Format for this script: 
## bash run_acousticoptim.sh INPUT_MESH SILHOUETTE_DEFORM_FLAG SILHOUETTE_REF_IMG ACOUSTIC_DEFORM_FLAG WHICH_ACOUSTIC_PARAMS OUTPUT_NAME
## where `INPUT_MESH_FILEPATH` is the absolute filepath of the mesh you would like to deform, 
##       `TRAINED_GRAPH_FILEPATH` is the filepath to the trained mesh2acoustic graph convolutional neural network,
##       `SILHOUETTE_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for a specific 2D silhouette, 
##       `SILHOUETTE_REF_IMG_FILEPATH` is the absolute filepath of the silhouette image you would like to use for mesh deformation, 
##       `SIL_WEIGHT` is a float number that weights the silhouette loss.
##       `CAMPOSITION_FILEPATH` is the absolute filepath of the text file that contains camera positions to render silhouette from mesh. 
##                           Note that each line of the file describes the location of one camera (e.g., two lines will yeild two cameras). 
##                           The location of each camera is described by its distance from the mesh, the elevation of the camera, and the angle from the x axis. 
##       `ACOUSTIC_DEFORM_FLAG` is either True or False depending on if you would like to deform the mesh to optimize for acostic quality, 
##       `ACOUST_DEFORM_WEIGHT` is a float number that weights the acoustic deform loss.
##       `WHICH_ACOUSTIC_PARAMS` is a string of the 10 acoustic quality parameters (each parameter needs to be separated by a comma with no spaces in between, see example below), 
##       `OUTPUT_FILENAME` is the absolute filepath where you would like the deformed output mesh to be saved.
##       `CFG_FILE` is the absolute path to the configuration file. You will need to change this to the config file path that you used for training the model (see above cell)
##
## Example Command:
#!cd /storage/concerthall && bash run_acousticoptim.sh \
#                            INPUT_MESH_FILEPATH \
#                            TRAINED_GRAPH_FILEPATH \
#                            SILHOUETTE_DEFORM_FLAG \
#                            SILHOUETTE_REF_IMG_FILEPATH \
#                            SIL_WEIGHT \
#                            CAMPOSITION_FILEPATH \
#                            ACOUSTIC_DEFORM_FLAG \
#                            ACOUST_DEFORM_WEIGHT \
#                            WHICH_ACOUSTIC_PARAMS \
#                            OUTPUT_FILENAME \
#                            CFG_FILE
#
!cd /storage/concerthall && bash run_acousticoptim.sh \
                            "/storage/3Dmodels/bench.obj" \
                            "/storage/concerthall_model_epoch99.pkl"\
                            "True" \
                            "/storage/2Dmodels/new000343.png" \
                            0.5 \
                            "/storage/camppos.txt" \
                            "True" \
                            10.0  \
                            '2660,24920,1.25,2066,17.7,57.7,0.54,0.31,9.37,1.55'  \
                            "test_sil_deform.obj" \
                            '/storage/concerthall/config/mesh2audioparams_train.yml'
